In [33]:
from langchain.prompts import ChatPromptTemplate

In [34]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch

In [35]:
llm = ChatOpenAI()

In [62]:
categorize_prompt = """
Given the following question about physics, classify it as 'beginner', 'expert' or 'uncategorized'.\n
Your answer should be based on the percieved level of the question. If it is basic (related to simpler physics topics), reply 'beginner'. If it relates to advanced physics topics reply 'expert'. If the question does not relate to physics, reply 'uncategorized'.\n
Do not respond with more than one word. \n
<question>
{question}
</question>
"""
chain = ChatPromptTemplate.from_template(template=categorize_prompt) | llm | StrOutputParser()

In [66]:
print(chain.invoke({"question": "what are feyman diagrams?"}))

expert


In [49]:
beginner_template="""You are a physics teacher who is really focused on beginners, explaining concepts simply. You assume no prior knowledge.
Here is your question: \n
{question}
"""

In [50]:
expert_template="""You are a physics professor who explains physics topics to advanced audience members. You can assume anyone you answer to has a PHD in Physics.
here is your question: \n
{question}"""

In [51]:
beginner_chain = ChatPromptTemplate.from_template(beginner_template) | llm
expert_chain = ChatPromptTemplate.from_template(expert_template) | llm

In [69]:
for s in beginner_chain.stream({"question": "what are magnets?"}):
    print(s.content, end="", flush=True)

Magnets are objects that have the ability to attract certain materials like iron, nickel, and cobalt. They can pull these materials towards them or push them away, depending on how they are positioned. Magnets have two ends or poles called the north pole and the south pole. Similar poles repel each other, meaning two north poles or two south poles will push away from each other, while opposite poles attract each other, meaning a north pole and a south pole will pull towards each other. This property of attracting or repelling other objects is known as magnetism. Magnets can be found naturally in certain rocks, but they can also be made by using special materials and processes.

In [52]:
general_prompt = ChatPromptTemplate.from_template(
    "You are a helpful assistant. Answer the question as accurately as you can.\n\n{input}"
)
branch = RunnableBranch(
  (lambda x: "beginner" in x["topic"].lower(), beginner_chain),
  (lambda x: "expert" in x["topic"].lower(), expert_chain),
  general_prompt
)

In [53]:
full_chain = {
    "topic": chain,
    "question": lambda x: x["question"]
} | branch

In [54]:
full_chain.invoke({"question": "what are magnets"})

AIMessage(content='Magnets are objects that have the ability to attract or repel certain materials. They have a special property called magnetism. When you think of magnets, you might picture those rectangular or horseshoe-shaped objects made of metal. But magnets can come in different shapes and sizes.\n\nWhat makes magnets special is that they create a magnetic field around them. This field is like an invisible force that reaches out from the magnet and can affect other objects. When two magnets are near each other, they can either pull towards each other or push away from each other, depending on how they are positioned. This is called attraction or repulsion.\n\nMagnets can also attract specific materials like iron, nickel, or cobalt. These materials are called magnetic materials. When a magnet and a magnetic material come close to each other, the magnetic field of the magnet makes the particles within the material align in a certain way, creating a temporary magnet. This is why ma

In [1]:
async for s in full_chain.astream({"question": "what are magnets?"}):
    print(s.content, end="", flush=True)

NameError: name 'full_chain' is not defined